In [ ]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import VGG19
import tensorflow.keras,os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, ZeroPadding2D, GlobalAveragePooling2D, DepthwiseConv2D
from tensorflow.keras import optimizers
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [ ]:
img_shape = (224, 224, 3)
model = VGG19(include_top=False, weights=None, input_shape=img_shape)
model.trainable = False

In [ ]:
model1 = ResNet101(include_top=False, weights=None, input_shape=img_shape)
model1.trainable = False

In [ ]:
tensorflow.keras.layers.concatenate([model, model1])

In [ ]:
model2 = Sequential([model, model1])
model2.add(ZeroPadding2D())
model2.add(DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='valid', depth_multiplier=1))
model2.add(GlobalAveragePooling2D())
model2.add(Flatten())
model2.add(Dense(units=1024,activation="relu"))
model2.add(Dense(units=1024,activation="relu"))
model2.add(Dense(units=1, activation="sigmoid"))
model2.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model2.trainable = False
model2.summary()

In [ ]:
model2 = ResNet101(include_top=False, weights=None, input_shape=img_shape)
model2.trainable = False

In [ ]:
model2 = Sequential([model1])


In [ ]:
model2.trainable = False
model2.summary()

In [ ]:
###########################################################
# 데이터 전처리
###########################################################


train_datagen = ImageDataGenerator(rescale=1./255)  # 모든 이미지를 1/255로 스케일을 조정합니다
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "./dataset/train2"
validation_dir = './dataset/vali2'

batch_size = 16
# batch_size = 64


train_generator = train_datagen.flow_from_directory( \
        train_dir,                  # 타깃 디렉터리
        target_size=(224, 224),     # 모든 이미지를 224 × 224 크기
        batch_size=batch_size,
        class_mode='binary')        #

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)  #(20, 150, 150, 3)
    print('배치 레이블 크기:', labels_batch.shape) #(20,)
    break

In [ ]:
steps_per_epochs = 50 # 학습데이터의 총 개수 / 배치 사이즈 (소수점 올림)
epochs = 30

history = model2.fit(
      train_generator ,
      steps_per_epoch=steps_per_epochs  ,
      epochs=epochs ,
      validation_data=validation_generator,
      validation_steps=18)

In [ ]:
###########################################################
# 학습 결과 정확도 및 비용 그래프
###########################################################
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')

plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model2.save("./tn_model/vgg_res_v3.h5")